In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
# リスト3.3.3 日本語用インデックスの登録
# ----------------------------------------------------------------------------------
# 目的:
#   ・Elasticsearch の日本語向け形態素解析（kuromoji）を用いた「検索用/索引用」アナライザを定義し、
#     専用インデックスを作成する最小例。
#
# 前提/注意:
#   ・本コードは「クライアント es（Elasticsearch インスタンス）」が既に存在する前提（例: リスト3.3.1）。
#   ・kuromoji/icu のプラグインや、サーバ側の形態素辞書/ユーザー辞書ファイルの配置が必要。
#     - "user_dictionary": "my_jisho.dic" は **ES ノード側** のパス（config ディレクトリ相対など）。
#     - Elastic Cloud など管理サービスではローカル辞書の直接配置ができない場合がある。
#   ・ここで定義するのは「アナライザ（analyzer）」まで。実運用では **マッピングで各 text フィールドに
#     index_analyzer / search_analyzer** を紐付けて初めて効果が出る（このスニペットでは作成のみ）。
#   ・同義語は初期空配列。運用開始後に更新したい場合はインデックス再作成 or チケット化（再オープン）などが必要。
#     - synonyms_filter はインデックス作成時に固定されやすく、ホットリロードは制約が多い点に注意。
# ----------------------------------------------------------------------------------

# インデックス作成用JSONの定義
create_index = {
    "settings": {
        "analysis": {
            # --- TokenFilter/CharFilter/Tokenizer/Analyzer を定義 ---
            "filter": {
                "synonyms_filter": {  # 同義語フィルタの定義（拡張は synonyms リストに語彙を列挙）
                    "type": "synonym",
                    # "expand": true が既定（A,B → A と B 双方向展開）。片方向にしたい場合は synonyms_graph も検討。
                    "synonyms": [  # 同義語リストの定義（ここでは空＝無効同然。後で再定義 or 再作成が必要）
                        # 例: "寿司, すし", "蕎麦, そば"
                    ],
                }
            },
            "tokenizer": {
                "kuromoji_w_dic": {  # カスタム形態素解析（kuromoji）にユーザー辞書を追加
                    "type": "kuromoji_tokenizer",  # ← ベースは kuromoji_tokenizer（コメントの綴りに注意）
                    # ユーザー辞書（ESノード側ファイル）。CSVではなく **ビルド済み dic** 指定を推奨。
                    # パスは ES 側の設定に依存（config ディレクトリ相対など）。存在しないとインデックス作成に失敗。
                    "user_dictionary": "my_jisho.dic",
                }
            },
            "analyzer": {
                # --- 検索時アナライザ: recall（想起）を高めるための同義語展開などを含める ---
                "jpn-search": {
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer",  # 文字正規化（NFKC 等）：全角/半角や合成文字の揺れを低減
                        "kuromoji_iteration_mark",  # 反復記号「ゝゞヽヾ」等の正規化
                    ],
                    "tokenizer": "kuromoji_w_dic",  # ユーザー辞書付き kuromoji
                    "filter": [
                        "synonyms_filter",  # 同義語展開（想起向上）。定義が空なら実質 no-op。
                        "kuromoji_baseform",  # 活用語の原形化（例: 食べた→食べる）
                        "kuromoji_part_of_speech",  # 品詞フィルタ（デフォルトの不要品詞を除去）。必要に応じて stoptags を明示的に設定推奨。
                        "ja_stop",  # 日本語ストップワード除去
                        "kuromoji_number",  # 数値正規化（桁区切り・漢数字の統一など）
                        "kuromoji_stemmer",  # 派生語の語幹化（長音の正規化など）
                    ],
                },
                # --- インデックス時アナライザ: precision（精度）寄り。通常は検索と分け、同義語は検索側で適用 ---
                "jpn-index": {
                    "type": "custom",
                    "char_filter": ["icu_normalizer", "kuromoji_iteration_mark"],
                    "tokenizer": "kuromoji_w_dic",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_part_of_speech",
                        "ja_stop",
                        "kuromoji_number",
                        "kuromoji_stemmer",
                    ],
                },
            },
        }
    }
}

# 日本語用インデックス名の定義
jp_index = "jp_index"

# 既存インデックスの削除（再作成時の衝突回避）
# - 本番では危険操作。検証用途に限定し、必ず対象を限定すること。
if es.indices.exists(index=jp_index):
    es.indices.delete(index=jp_index)

# インデックス jp_index の生成
# - ここでは settings（analysis）のみ定義。実運用では mappings で text フィールドに
#   "analyzer": "jpn-index", "search_analyzer": "jpn-search" を **紐付け** すること。
es.indices.create(index=jp_index, body=create_index)

# 【補足：運用ベストプラクティス（コメントのみ）】
# - 同義語更新: synonyms を変えたい場合は、別インデックスを作成→エイリアス切替が最も安全（ゼロダウンタイム）。
#   どうしても差し替えたい時は、close→settings update（再open）や synonyms_graph の利用を検討。
# - 検索確認: _analyze API でトークンを確認（例: POST jp_index/_analyze { "analyzer": "jpn-index", "text": "お寿司が食べたい" }）。
# - マッピング例（参考・ここでは適用しない）:
#   {
#     "mappings": {
#       "properties": {
#         "title":   { "type": "text", "analyzer": "jpn-index", "search_analyzer": "jpn-search" },
#         "content": { "type": "text", "analyzer": "jpn-index", "search_analyzer": "jpn-search" }
#       }
#     }
#   }

In [ ]:
# リスト 3.3.4 分析結果表示用関数 analyse_jp_test
# ----------------------------------------------------------------------------------
# 目的:
#   ・前段で作成した日本語用インデックス（jp_index）に紐づくアナライザ "jpn-search" を使い、
#     与えた文字列を Elasticsearch の _analyze API でトークン化し、その「語彙列（token 文字列）」のみを返す。
#
# 理論メモ:
#   ・indices.analyze(index=..., analyzer=..., text=...) は、指定インデックスの settings.analysis
#     に従って「char_filter → tokenizer → filter」を適用した“検索側の前処理”をシミュレートする。
#   ・ここで指定している "jpn-search" は kuromoji をベースに、
#       - 文字正規化（icu_normalizer, iteration_mark）
#       - 形態素解析（kuromoji_tokenizer + user_dictionary）
#       - 原形化/品詞フィルタ/ストップワード/数字正規化/ステミング
#       - 同義語展開（synonyms_filter；現在は空定義）
#     といったチェーンで語彙（token）を生成する想定。
#   ・返却される ret["tokens"] は辞書リストで、各要素は
#       {"token": "語彙", "start_offset": 0, "end_offset": 2, "type": "...", "position": 0}
#     のような構造。実務では offset/position を保持するとデバッグが楽。
#
# 注意/運用:
#   ・"jpn-search" はインデックス jp_index の settings に存在している必要がある（なければ 400 エラー）。
#   ・アナライザ定義（同義語やユーザー辞書）を変えた場合は再インデックス or エイリアス切替が基本。
#   ・学習用途では可視性のため token 文字列のみ返しているが、下流でハイライト/解析をするなら
#     位置情報（position, start_offset, end_offset）も返す設計がよい。
# ----------------------------------------------------------------------------------


def analyse_jp_text(text):
    # _analyze API へのリクエストボディを組み立て
    # - analyzer: 使用するアナライザ名（ここでは検索時用 "jpn-search"）
    # - text    : 解析対象の文字列
    body = {"analyzer": "jpn-search", "text": text}

    # 指定インデックス（jp_index）のアナライザを使って解析を実行
    # - インデックスを指定することで、その index 専用の tokenizer/filter/辞書設定が反映される
    ret = es.indices.analyze(index=jp_index, body=body)

    # レスポンスから tokens 配列（各トークンの辞書）を取り出し、
    # "token" フィールド（実際の語彙文字列）だけを抽出して返す
    tokens = ret["tokens"]
    tokens2 = [token["token"] for token in tokens]
    return tokens2


# analyse_jp_test 関数のテスト
# - 例: 「関数のテスト」→ ["関数","テスト"] など（実際の出力は辞書/フィルタ設定に依存）
print(analyse_jp_text("関数のテスト"))

# 参考: 位置情報も見たい場合（デバッグ用; 実行例）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"関数のテスト"})["tokens"]
# for t in toks:
#     print(t["token"], t["position"], t["start_offset"], t["end_offset"])

In [ ]:
# リスト 3.3.5 icu_normalizerのテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・analysis で設定した `icu_normalizer`（文字正規化）と kuromoji の組み合わせによって、
#     半角カナや互換文字が正規化され、同一の語彙（トークン）として扱われることを確認する。
#
# 理論メモ:
#   ・`icu_normalizer` は Unicode の互換正規化（NFKC 系）とケース折り畳み等により、
#     全角/半角・互換文字（例: ㌀ などの「平方化（square）」表現）を正規形に写像する。
#   ・本インデックスの `jpn-search` は `char_filter` → `tokenizer(kuromoji)` → `filter` の順。
#     このため **正規化は形態素解析の前** に適用され、トークン化の安定性（分割・語形）を高める。
#   ・テスト文字列:
#       - 'ｱﾊﾟｰﾄ' : 半角カナ + 長音記号 → 正規化で全角カタカナ『アパート』相当へ
#       - '㌀'     : 互換文字（SQUARE APAATO）→ 正規化で『アパート』相当へ
#     期待される結果はいずれも「アパート」一語のトークン列（例: ["アパート"]）。
#     ※ 実際の出力は辞書/フィルタ構成・ユーザー辞書に依存するが、少なくとも同一化されるのが狙い。
#
# 前提:
#   ・`analyse_jp_text` は前リスト（3.3.4）で定義済みで、`jpn-search` アナライザを使って _analyze する関数。
#   ・`jp_index` が存在し、その settings.analysis に `jpn-search` が定義されていること。
# ----------------------------------------------------------------------------------

# 半角カナ → 正規化 → 形態素解析 → トークン列
print(analyse_jp_text("ｱﾊﾟｰﾄ"))

# 互換文字（平方化記号） → 正規化 → 形態素解析 → トークン列
print(analyse_jp_text("㌀"))

In [ ]:
# リスト 3.3.6 kuromoji_iteration_markのテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・`jpn-search` の char_filter に指定した `kuromoji_iteration_mark`（反復記号の正規化）が
#     形態素解析前に「々/ゝ/ゞ/ヽ/ヾ」等を直前文字へ展開し、語彙の揺れを吸収することを確認する。
#
# 理論メモ:
#   ・日本語には「々（踊り字）」「ゝ/ゞ（ひらがな繰返し）」「ヽ/ヾ（カタカナ繰返し）」があり、
#     書記上は同一語でも文字列としては別形になる。これらは **トークン化前** に正規化するのが安定。
#   ・`kuromoji_iteration_mark` は char_filter として働き、例:
#       - 「時々」→「時時」
#       - 「こゝろ」→「こころ」
#       - 「すゝめ」→「すすめ」
#     のように展開してから `kuromoji_tokenizer` へ渡すため、辞書ヒット率・語形一貫性が向上する。
#   ・本インデックスの `jpn-search` は
#       char_filter（icu_normalizer, kuromoji_iteration_mark）→ tokenizer（kuromoji）→ filter
#     の順に適用されるため、反復記号の処理は必ず形態素解析より先に行われる。
#
# 前提:
#   ・analyse_jp_text は 3.3.4 で定義済み（_analyze API で "jpn-search" を適用して token 文字列配列を返す）。
#   ・jp_index の settings.analysis.analyzer に "jpn-search" が存在すること。
# ----------------------------------------------------------------------------------

# 例1: 踊り字「々」を展開（期待: 「時々」→「時時」→ token 化で安定化）
print(analyse_jp_text("時々"))

# 例2: ひらがな繰返し記号「ゝ」を展開（期待: 「こゝろ」→「こころ」）
print(analyse_jp_text("こゝろ"))

# 例3: ひらがな繰返し記号「ゝ」を含む熟語を展開（期待: 「すゝめ」→「すすめ」）
print(analyse_jp_text("学問のすゝめ"))

# （参考: 期待値の一例／環境依存。辞書とフィルタにより結果は異なる）
#  - '時々'          -> ['時々'] または ['時', '時']（辞書により表層保持か分割かが変わる）
#  - 'こゝろ'        -> ['こころ']
#  - '学問のすゝめ'  -> ['学問','すすめ']
# 実運用では _analyze の tokens から position / offset も併せてログ出力し、可視デバッグを推奨。

In [ ]:
# リスト 3.3.7 kuromoji_baseform のテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" のフィルタ列に含まれる `kuromoji_baseform`（原形化）が
#     動詞・形容詞などの「活用形」を**辞書上の基本形（原形）**に正規化することを確認する。
#
# 理論メモ:
#   ・本インデックスの "jpn-search" は、char_filter → tokenizer(kuromoji) → filter の順で適用される。
#   ・`kuromoji_baseform` は **tokenizer の後** に働き、各トークンが持つ「原形」属性へ置き換える。
#       例）「飲みに行った」→ 形態素分割後に
#            「飲み（連用形）」→『飲む』, 「行った（過去形）」→『行く』 へ正規化される。
#   ・その後に `kuromoji_part_of_speech` や `ja_stop` が続くため、助詞「に」や読点などは
#     **品詞/ストップワード除去**で落ちる構成（= 検索時の雑音低減）。
#
# 期待される出力（環境依存の一例）:
#   ・['昨日', '飲む', '行く']
#     - 「昨日」……名詞（そのまま）
#     - 「飲み」… 動詞『飲む』へ原形化
#     - 「行った」… 動詞『行く』へ原形化
#     - 助詞「に」や句読点はフィルタで除去される想定
#
# 前提:
#   ・analyse_jp_text(text) は 3.3.4 で定義済みで、インデックス jp_index の "jpn-search" で _analyze を実行し
#     token 文字列だけの配列を返す関数。
# ----------------------------------------------------------------------------------

print(analyse_jp_text("昨日、飲みに行った。"))

In [ ]:
# リスト 3.3.8 kuromoji_part_of_speech のテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" のフィルタ列に含まれる `kuromoji_part_of_speech`
#     （品詞フィルタ）が、機能語（助詞・助動詞・記号 等）を除去して**内容語中心**の
#     語彙列へ整形する効果を確認する。
#
# 理論メモ:
#   ・処理順は「char_filter → tokenizer(kuromoji) → filter」の順。
#   ・`kuromoji_part_of_speech` は **tokenizer の後** に動作し、所定の品詞（stoptags）に
#     マッチするトークンを削除する。
#     - 典型的な既定: 「助詞」「助動詞」「記号」等（実装の既定リストに依存）。
#     - `ja_stop` と併用することで、頻出の機能語や指示詞なども追加的に除去される。
#   ・本例「この店は寿司がおいしい。」では、次のような挙動が期待される（環境依存の一例）:
#       - 残る: 「店」「寿司」「おいしい」（内容語）
#       - 落ちる: 「この」（連体詞/指示詞: ja_stop/POS フィルタで除去され得る）、
#                 「は」「が」（助詞）、句点「。」（記号）
#     → 期待される出力例: ['店', '寿司', 'おいしい']
#
# 備考:
#   ・どの品詞が除去対象かは `kuromoji_part_of_speech` の `stoptags` 設定で調整可能。
#     精度/再現性のため、プロダクションでは **stoptags を明示** しておくことを推奨。
#   ・出力は辞書やフィルタ構成に依存するため、厳密な検証時は _analyze の詳細（position/offset/type）
#     も併せてログ出力するとよい。
print(analyse_jp_text("この店は寿司がおいしい。"))

In [ ]:
# リスト 3.3.9 ja_stop のテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" に含まれる `ja_stop`（日本語ストップワード辞書）が、
#     指示詞・機能語などの高頻出語を除去して「検索に有用な内容語」へ寄せる効果を確認する。
#
# 理論メモ:
#   ・"jpn-search" の処理順は
#       char_filter（icu_normalizer, kuromoji_iteration_mark）
#       → tokenizer（kuromoji）
#       → filter（synonyms, kuromoji_baseform, kuromoji_part_of_speech, ja_stop, kuromoji_number, kuromoji_stemmer）
#     で適用される。
#   ・`ja_stop` は **tokenizer 後** の語彙列から、あらかじめ定義された日本語ストップワード（機能語・指示語 など）
#     を削除する Lucene 由来のフィルタ。これにより、ランキングでノイズになりやすい語を抑制できる。
#   ・本例「しかし、これでいいのか迷ってしまう。」の処理イメージ（環境依存の一例）:
#       - 原形化: 「いい」→「良い」, 「迷って」→「迷う」, 「しまう」→（助動）→ POS/stop で除去され得る
#       - 品詞/ストップ除去: 「しかし」「これ」「で」「か」等は `ja_stop` や POS フィルタで削除対象になりやすい
#       → 期待される出力例: ['良い', '迷う'] あるいは ['迷う']（辞書・stoptags 構成に依存）
#     ※ 厳密な出力はインデックスの設定（`kuromoji_part_of_speech` の stoptags、`ja_stop` のリスト）に左右される。
#
# デバッグの勧め:
#   ・トークン文字列だけでなく position / offset も見ると、どの段で落ちたか原因追跡が容易。
#   ・差分検証は `_analyze` を「ja_stop あり/なし」のアナライザで比較するのが有効。
# ----------------------------------------------------------------------------------

print(analyse_jp_text("しかし、これでいいのか迷ってしまう。"))

# 参考: 位置情報も確認したい場合（コメントアウトのまま利用）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"しかし、これでいいのか迷ってしまう。"})["tokens"]
# for t in toks:
#     # 例: token='迷う', pos=*, start/end は原文中のオフセット
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))

In [ ]:
# リスト 3.3.10 kuromoji_number のテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" に含まれる `kuromoji_number`（数値正規化フィルタ）が、
#     日本語の数表現（漢数字＋位取り記号「十・百・千・万・億・兆」等）を **アラビア数字へ正規化**
#     し、検索時の揺れを吸収できることを確認する。
#
# 理論メモ:
#   ・処理順は「char_filter（icu_normalizer 等）→ tokenizer（kuromoji）→ filter（…，kuromoji_number）」。
#   ・`kuromoji_number` は tokenizer 後のトークン列に対し、数表現の連なりを解析して 1 つの
#     数値トークンへ正規化/集約する（例: 「二十三」→ "23"、 「一億二十三」→ "100000023"）。
#   ・利点: 数表記の揺れ（漢数字/算用数字/全角半角/位取りの有無）を統一でき、集計・一致が安定する。
#   ・注意: 位取りや書法が曖昧な入力（例: 「一〇〇万」など）も多い。辞書・フィルタ順・他フィルタ
#     （ja_stop, POS フィルタ等）との相互作用で結果が変わり得るため、_analyze での期待出力を
#     事前に確認しておくこと。
#
# 期待される出力（環境依存の一例）:
#   ・analyse_jp_text('一億二十三') -> ['100000023']
#     （100,000,000 + 23 = 100,000,023）
#
# 前提:
#   ・analyse_jp_text は 3.3.4 で定義済み（"jpn-search" を用いて _analyze し、token 文字列配列を返す）。
#   ・jp_index の settings.analysis.analyzer に "jpn-search" が存在し、filter に kuromoji_number が含まれる。
# ----------------------------------------------------------------------------------

print(analyse_jp_text("一億二十三"))

# （追加テスト例：必要に応じて実行）
# print(analyse_jp_text('二千二十四'))     # 例: ['2024']
# print(analyse_jp_text('一万二千三百四十五')) # 例: ['12345']
# print(analyse_jp_text('一・二・三'))       # 正規化のされ方は設定に依存

In [ ]:
# リスト 3.3.11 kuromoji_stemmer のテスト
# ----------------------------------------------------------------------------------
# 目的:
#   ・検索用アナライザ "jpn-search" のフィルタ列に含まれる `kuromoji_stemmer`
#     （カタカナの長音正規化）が、カタカナ語末尾の長音記号「ー」を取り除いて
#     語形の揺れを吸収する効果を確認する。
#
# 理論メモ:
#   ・"jpn-search" の処理順（本リポの定義に従う）:
#       char_filter（icu_normalizer, kuromoji_iteration_mark）
#       → tokenizer（kuromoji + user_dictionary）
#       → filter（synonyms_filter → kuromoji_baseform → kuromoji_part_of_speech
#                  → ja_stop → kuromoji_number → **kuromoji_stemmer**）
#   ・`kuromoji_stemmer` は **最後段** で適用され、主に **カタカナ語**に対して
#     長音記号「ー」を削減（正規化）する。例: 「コンピューター」→「コンピュータ」。
#     これにより「表記の長さ揺れ」を同一語としてマッチさせやすくなる。
#   ・本例文「コンピューターを操作する」の処理イメージ（環境依存の一例）:
#       1) 形態素分割: 「コンピューター」「を」「操作」「する」
#       2) 原形化     : 名詞はそのまま／動詞「する」は原形「する」
#       3) 品詞/Stop  : 助詞「を」は除去（POS/ja_stop）
#       4) 長音正規化: 「コンピューター」→「コンピュータ」
#     → 期待されるトークン列: ['コンピュータ', '操作', 'する']
#     ※ 実際の出力は辞書・stoptags・ユーザー辞書設定に依存する。
#
# 運用のヒント:
#   ・「コンピュータ」「コンピューター」「コンピュータ－」等の表記差を横断して検索したい場合に有効。
#   ・ブランド名等で長音の有無が**区別的意味**を持つときは、当該フィールドでの stemmer 適用を避ける（別フィールドを用意）。
# ----------------------------------------------------------------------------------

print(analyse_jp_text("コンピューターを操作する"))